## Exercise 3

### Descrição do Problema

O objetivo deste exercício é desenvolver uma AEAD com "Tweakable Block Ciphers”", 
sendo a cifra usada do tipo AES-256 ou o ChaCha20.

Esta cifra será usada num canal privade de informação assíncrona com acordo de chaves.
A autenticação dos agentes será com:
 - “X448 key exchange”
 - “Ed448 Signing&Verification” 
 
... e deve haver uma fase de confirmação da chave acordada.


Ou seja,

Use o “package” Cryptography para
1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

In [1]:
#Imports necessários
import os
import asyncio
import random

from pickle import dumps, loads
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import x448
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey

# Para a porção de código associada com as queues (na comunicação entre participantes)
import nest_asyncio

nest_asyncio.apply()

## Criação das chaves privadas/públicas

O acordo de chaves, como indicado no enunciado, deve ser feito com __"X448 key exchange"__.

Para tal é necessário que cada um dos agentes criem as suas próprias chaves (privada-pública).

Neste caso será usando a __curva448__, como 
apresentado na API:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/x448/

A chave resultante será usada no decorrer da comunicação assíncrona entre agentes para __cifrar/decifrar__ os dados.

In [2]:
def generateKeys():
    # Generate private key for exchange
    prv_key = x448.X448PrivateKey.generate()
    
    # Generate public key thorugh private key
    pub_key = prv_key.public_key()
    
    return prv_key, pub_key
    

## Criação das chaves partilhadas

Após ter as chaves de cada agente (privada/pública), podemos estabelecer
o _exchange_, i.e o acordo entre ambos agentes,
sobre um segredo partilhado.

A _shared_key_ criada, tal como recomendado na API, deve ser passada por uma função de derivação, no âmbito de a tornar mais segura; adicionando mais informações à chave para destruir qualquer estrutura que possa ser criada.

Será criada uma _shared_key_, a ser usada no âmbito da cifra. 

In [3]:
def generateShared(prv_key, peer_key):
    
    peer_cipher_key = x448.X448PublicKey.from_public_bytes(peer_key)
    
    # Gerar uma chave partilha para cifra
    cipher_key = prv_key.exchange(peer_cipher_key)
    
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(cipher_key)
    
    return cipher_key

## Assinar mensagem

A autenticação dos agentes é realizada com 
__“Ed448 Signing&Verification”__; sendo este um 
algoritmo de assinaturas com recurso ao __EdDSA__ (_Edwards-curve Digital Signature Algorithm_).

Para tal é necessário um novo par de chaves (privada/pública), do tipo _Ed448_, para cada um dos agentes.

Para recorrer aos meios apresentados em:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed448/

As chaves públicas devem ser partilhadas por ambos, de 
modo a validar a __assinatura__ da mensagem que receberam.
Sendo a assinatura realizada através da chave privada do agente,
a enviar a mensagem.

Criou-se um método particular para gerar as chaves de autenticação dos agentes, para facilitar e tornar o código mais _readable_.

In [4]:
def generateSignKeys():
    
    ## Chave privada para assinar
    private_key = Ed448PrivateKey.generate()
    
    ## Chave pública para autenticar
    public_key = private_key.public_key()
   
    return private_key, public_key
    
def signMsg(prv_key, msg):
    
    signature = prv_key.sign(msg)
    
    return signature

## Cifrar mensagem

In [5]:
def pad (last_block, len_last_block , length):
    print("Len Last Block: "+str(len_last_block))
    
    for _ in range (len_last_block, length):
        last_block += "0"
    
    print("Last Block: "+last_block)
    print("New Len Last Block: "+str(len(last_block)))
    
    return last_block
    
def tpbc (tweak, key, block):
    tweaked_key = tweak ++ bytes(key, 'utf-8')
    print(len(tweaked_key))
    cipher = Cipher(algorithms.AES256(tweaked_key), mode = None)
    encryptor = cipher.encryptor()
    ct_block = encryptor.update(block) + encryptor.finalize()
    print(ct_block)
    return ct_block

def encrypt(msg, key):
    
    print("Plaintext: "+msg)
    NBytes = 16; size_nounce = 8
    
    digest = hashes.Hash(hashes.SHA3_256())
    nounce_temp = digest.finalize()
    
    nounce = nounce_temp[:size_nounce]
    print("Nounce: "+ str(nounce))
    
    ctr = os.urandom(size_nounce)
    print("CTR: "+str(ctr)) # tirar um bit ao ctr
    
    block_plaintext = [msg[i:i+NBytes] for i in range(0, len(msg), NBytes)]    
    
    last_block = block_plaintext[-1]
    len_last_block = len(last_block)     
    block_plaintext[-1] = pad(last_block, len_last_block, NBytes)
    
    # i= 0 ... m - 1    
    ct = ""
    auth = ""
    
    print(block_plaintext)
    
    for elem in (block_plaintext[:-1]):
        tweak = nounce ++ ctr ++ b'0'
        print(tweak)
        #c_i = tpbc(tweak, key, elem)
        #ct += c_i
        ctr += 1
        #auth = auth ^ elem    
    
    '''
    
    # i = m
    tweak = nounce ++ ctr ++ b'0'
    c_aux = tpbc(tweak, key, len_last_block)
    c_m = last_block ^ c_aux
    ct += c_m
    
    auth = auth ^ c_m
    
    # Autenticação
    
    tweak = nounce ++ ctr ++ b'1'
    
        
    
    tag = tpbc(tweak, key, auth)
    
    return {"ct": ct, "tag": tag, "nounce": nounce, "ctr": ctr, "pad": len_last_block}
    
    '''

## Decifrar mensagem

In [6]:
# size_block = tau
def unpad(last_block, size_block, NBytes):
    
    for _ in range (size_block, NBytes):
        last_block -= 0
    
    return last_block

def un_tpbc(tweak, key, block):
    tweaked_key = tweak ++ key
    cipher = Cipher(algorithms.AES256(tweaked_key), mode = None)
    decryptor = cipher.decryptor()
    plain_block = decryptor.update(block) + decryptor.finalize()
    return plain_block

def decrypt(msg, key):
    ct = msg['ct']
    tag_rcv = msg['tag']
    nounce = msg['nounce']
    ctr = msg['ctr']
    len_last_block = msg["pad"]
    
    NBytes = 16
    
    block_ciphertext = [ct[i:i+NBytes] for i in range(0, len(ct), NBytes)]    
        
    # i= 0 ... m - 1
    plaintext = ""
    auth = ""
    for elem in block_ciphertext[:-1]:
        tweak = nounce ++ ctr ++ b'0'
        c_i = tpbc(tweak, key, elem)
        plaintext += c_i
        auth = auth ^ c_i
        ctr += 1
    
    # i = m
    tweak = nounce ++ ctr ++ b'0'
    c_aux = tpbc(tweak, key, len_last_block)
    c_m = last_block ^ c_aux
    auth = auth ^c_m
    
    # Autenticação
    tweak = nounce ++ ctr ++ b'1'
    tag = tpbc(tweak, key, auth)
    
    if tag != tag_rcv:
        print("Mensagem inválida")
    
    last_block = unpad ( c_m , len_last_block , NBytes)

## Funções auxiliares para o canal de comunicação

In [7]:
def init_comm_emitter():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    
    prv_sign_key, pub_sign_key = generateSignKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    ), 
           'sign_key': pub_sign_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    )
            }
    
    return prv_cipher_key, pub_cipher_key, prv_sign_key, pub_sign_key, msg

def init_comm_receiver():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    )
          } 
    
    return prv_cipher_key, pub_cipher_key, msg

async def send(queue, msg):
    
    await asyncio.sleep(random.random())
        
    # put the item in the queue
    await queue.put(msg)
    
    await asyncio.sleep(random.random())
    
async def receive(queue):
    item = await queue.get()

    await asyncio.sleep(random.random())
    aux = loads(item)

    return aux

## Emitter

In [8]:
## Emitter Code
async def emitter(plaintext, queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, pub_cipher_key, prv_sign_key, pub_sign_key, msg = init_comm_emitter()
    
    ## Enviar a chaves públicas para o peer
    await send(queue, dumps(msg))
    print("[E] SENDING PUBLIC KEYS")
    
    ## Receber as chaves públicas do peer
    msg = await receive(queue)
    print("[E] RECEIVED PEER PUBLIC KEYS")
    
    pub_peer_cipher = msg['cipher_key']
    print("[E] Receiver pub_key_cipher: " +str(msg['cipher_key']))
    
    ## Criar as chaves partilhadas (cifrar/autenticar)
    cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)
    
    
    ## Cifrar a mensagem
    pkg = encrypt(plaintext, cipher_shared)
    print("[E] MESSAGE ENCRYPTED")
    
    ## Assinar e enviar a mensagem
    pkg_b = dumps(pkg)
    sig = signMsg(prv_cipher_key, pkg_b)
    
    # Enviar
    msg_final = {'sig': sig, 'msg': dumps(pkg)}
    
    print("[E] SENDING MESSAGE")
    await send(queue, dumps(msg_final))
    
    print("[E] END")

## Receiver

In [9]:
## Receiver Code
async def receiver(queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, pub_cipher_key, msg = init_comm_receiver()
    
    ## Receber as chaves publicas do peer
    pub_keys = await receive(queue)
    
    pub_peer_cipher = pub_keys['cipher_key']
    pub_peer_sign = pub_keys['sign_key']
    
    print("[R] Emitter pub_key_cipher: " +str(pub_peer_cipher))
    print("[R] Emitter pub_key_sign: " +str(pub_peer_sign))
    
    ## Gerar shared keys
    cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)

    ## Enviar as chaves públicas ao peer
    await send(queue, dumps(msg))
    print("[R] AWAIT CIPHER")
    ciphertext = await receive(queue)
    print("[R] CIPHER RECEIVED")
    
    ## Receber a mensagem (Assinatura)
    peer_cipher_key = x448.X448PublicKey.from_public_bytes(pub_peer_cipher)
    
    ## Validar a correçaõ da assinatura
    peer_public_key.verify(ciphertext['sig'], ciphertext['msg'], ec.ECDSA(hashes.SHA3_256()))
       
    msg_dict = loads(ciphertext['msg'])
                      
    ## Decifrar essa mensagem       
    plain_text = decrypt()
    
    ## Apresentar no terminal
    print("[R] Plaintext: " + plain_text.decode('utf-8'))
       

In [10]:
def test(msg):
    loop = asyncio.get_event_loop()
    queue = asyncio.Queue(10)
    asyncio.ensure_future(emitter(msg, queue), loop=loop)
    loop.run_until_complete(receiver(queue))
    # emitter(msg)

test("HELLO WORLD! THIS IS JUST A TEST OF A AEAD ALGORITHM.")

[R] Emitter pub_key_cipher: b'\xb3R\x9bR\x1a\xe6L\x04\xe7\xc1D\x81`Ts\xfd\xd9{`(P\xa7\x82\xd3W\x07Av|:U\x02wt\x94\xfe\xfc\x86\xc9\xc5\xc6o\xaa\xac/\xa8\xa4j\xdeq\xb5\x1b\x8b\x9c\x8b\xa7'
[R] Emitter pub_key_sign: b'\xda+Vo\x1fk\xdfo\xe4O9hb\'\xb4_\xdfmz;\xe7"\x8cu\x9f\x0c\xd0e\x00\xb2\xbc\xca\r\xedx\xf3\xa6\x08\x81\x13g8L?|\xd7\xa5\xc7=\xeaf\x17\x83\xfd\xee\x99\x80'
[E] SENDING PUBLIC KEYS
[R] AWAIT CIPHER
[E] RECEIVED PEER PUBLIC KEYS
[E] Receiver pub_key_cipher: b'!\xac\xc2[\x1aP\xf2b\xd9t\x1d\xb2\xd3\xe65o\x18\xee\xe3\xc3\xcc\xfa;\xae\xaf\x8bHM\x99\xdf6\xc7\xe3\xb8\xfa2\xce\xae\x82\xe9>Q\xc6Q\x9c\xe5\xcaW<\x82\xb2%\xcd\xe6\xb8k'
Plaintext: HELLO WORLD! THIS IS JUST A TEST OF A AEAD ALGORITHM.
Nounce: b'\xa7\xff\xc6\xf8\xbf\x1e\xd7f'
CTR: b'p\x126E\xd7\xfe\x10f'
Len Last Block: 5
Last Block: ITHM.00000000000
New Len Last Block: 16
['HELLO WORLD! THI', 'S IS JUST A TEST', ' OF A AEAD ALGOR', 'ITHM.00000000000']


KeyboardInterrupt: 